In [1]:
import pandas as pd
import numpy as np

xlsx_path = r"source_data/Тестовое_задание_Data_аналитик_ЦО_2025_.xlsx"
df = pd.read_excel(xlsx_path, sheet_name="Данные для задачи 1")
rules = pd.read_excel(xlsx_path, sheet_name="Справочник наценок")

df.shape, rules.shape

((683, 19), (123, 8))

In [2]:
base_cols = [
    "Бренд","Код товара","Наименование товара",
    "Категория1","Категория2","Категория3","Категория4",
    "Сегмент","Цена закупки товара"
]
competitor_cols = [c for c in df.columns if c not in base_cols]

df["Код товара"] = pd.to_numeric(df["Код товара"], errors="coerce").astype("Int64")
df["cost"] = pd.to_numeric(df["Цена закупки товара"], errors="coerce").astype(float)
for c in competitor_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

assert df["Код товара"].notna().all() and (df["cost"] > 0).all()
competitor_cols

['letu.ru',
 'iledebeaute.ru',
 'rivegauche.ru',
 'Ozon',
 'Wildberries',
 'MagnitMarket',
 'Megamarket',
 'lamoda.ru',
 'tsum.ru',
 'visagehall.ru']

In [3]:
rules = rules.copy()
rules["Активный"] = rules["Активный"].astype(str).str.strip().str.lower()
rules = rules[rules["Активный"] == "да"].copy()

rules["Продукт"] = pd.to_numeric(rules["Продукт"], errors="coerce").astype("Int64")
for col in ["Сегмент рынка","Категория","Бренд"]:
    rules[col] = rules[col].astype("string").str.strip()
rules["Наценка"] = pd.to_numeric(rules["Наценка"], errors="coerce").astype(float)

rules.head()

,Сегмент рынка,Категория,Бренд,Продукт,Дата начала,Дата окончания,Наценка,Активный
0,Люкс,<NA>,<NA>,<NA>,2025-01-01,NaT,1.2,да
1,Массмаркет,<NA>,<NA>,<NA>,2025-01-01,NaT,0.7,да
2,Люкс,Макияж,<NA>,<NA>,2025-01-01,NaT,0.9,да
3,Люкс,Уход,<NA>,<NA>,2025-01-01,NaT,1.5,да
4,Массмаркет,Макияж,<NA>,<NA>,2025-01-01,NaT,0.5,да


## Расчет топ 3 сайта и получение наценки по приоритетам

In [4]:
def top3_prices_with_sites(row, cols):
    """
    Получаем три минимальные цены и соответствующие им сайты.
    """
    s = row[cols].dropna().sort_values()
    if s.empty:
        return pd.Series([np.nan, None, np.nan, None, np.nan, None])
    values_arr, site_names = s.values, list(s.index)
    min1, site1 = float(values_arr[0]), str(site_names[0])
    min2, site2 = (float(values_arr[1]), str(site_names[1])) if len(values_arr) > 1 else (np.nan, None)
    min3, site3 = (float(values_arr[2]), str(site_names[2])) if len(values_arr) > 2 else (np.nan, None)
    return pd.Series([min1, site1, min2, site2, min3, site3])


def pick_markup(row, rules):
    """
    Получаем наценку по прироритетам:
    1. Продукт
    2. Бренд/Категория3
    3. Бренд/Категория2
    4. Бренд/Категория1
    5. Бренд
    6. Сегмент/Категория3
    7. Сегмент/Категория2
    8. Сегмент/Категория1
    9. Сегмент
    """
    sku = int(row["Код товара"])
    brand = str(row["Бренд"]).strip()
    seg = str(row["Сегмент"]).strip()
    c1 = str(row["Категория1"]).strip()
    c2 = str(row["Категория2"]).strip()
    c3 = str(row["Категория3"]).strip()

    def mask(col, val):
        return rules[col].isna() if val is None else rules[col].eq(val)

    priorities = [
        ("Продукт",{"Продукт": sku}),
        ("Бренд/Категория3",{"Бренд": brand, "Категория": c3}),
        ("Бренд/Категория2",{"Бренд": brand, "Категория": c2}),
        ("Бренд/Категория1",{"Бренд": brand, "Категория": c1}),
        ("Бренд",{"Бренд": brand, "Категория": None, "Сегмент рынка": None}),
        ("Сегмент/Категория3",{"Сегмент рынка": seg, "Категория": c3}),
        ("Сегмент/Категория2",{"Сегмент рынка": seg, "Категория": c2}),
        ("Сегмент/Категория1",{"Сегмент рынка": seg, "Категория": c1}),
        ("Сегмент",{"Сегмент рынка": seg, "Категория": None, "Бренд": None}),
    ]

    for level_name, conditions in priorities:
        m = pd.Series(True, index=rules.index)
        for col, val in conditions.items():
            m &= mask(col, val)
        sub = rules[m]
        if not sub.empty:
            return float(sub["Наценка"].max()), level_name

    return np.nan, "Не найдено"


df[["min1","min1_site","min2","min2_site","min3","min3_site"]] = df.apply(lambda r: top3_prices_with_sites(r, competitor_cols), axis=1)
picked = df.apply(lambda r: pick_markup(r, rules), axis=1, result_type="expand")
df["markup"] = picked[0].astype(float)
df["markup_rule"] = picked[1]

assert df["min1"].notna().all() and df["markup"].notna().all()
df[["Код товара","Бренд","markup","markup_rule","min1","min1_site","min2","min2_site","min3","min3_site"]].head()


,Код товара,Бренд,markup,markup_rule,min1,min1_site,min2,min2_site,min3,min3_site
0,19000412437,Clarins,1.03,Бренд,1029.0,iledebeaute.ru,1117.0,lamoda.ru,NaN,None
1,14037100013,Clarins,1.03,Бренд,2107.0,iledebeaute.ru,NaN,None,NaN,None
2,19000033387,Clarins,1.03,Бренд,919.0,rivegauche.ru,924.0,lamoda.ru,964.0,Ozon
3,14037100015,Clarins,1.03,Бренд,2355.0,iledebeaute.ru,3536.0,rivegauche.ru,NaN,None
4,19000168458,Clarins,1.03,Бренд,924.0,iledebeaute.ru,1039.0,rivegauche.ru,1045.0,Ozon


## Расчёт цены до скидки, итоговой цены продажи и статуса (top‑3)

In [5]:
# Цена до скидки (целая) и минимальная цена для маржи 10%
df["base_price"] = np.round(df["cost"] * (1 + df["markup"])).astype(int)
df["min_price_10pct"] = np.ceil(df["cost"] / 0.9).astype(int)

# Цели по рынку (целые рубли): строго дешевле 1/2/3 минимума
df["tgt1"] = (df["min1"] - 1).round().astype(int).clip(lower=1)
df["tgt2"] = (df["min2"] - 1).round().astype("Int64")
df["tgt3"] = (df["min3"] - 1).round().astype("Int64")


def pick_price_and_discount(base_price, min_price_10pct, targets, labels):
    """
    Выбор цены и скидки по приоритетам целей
    """
    for tgt, label in zip(targets, labels):
        if pd.isna(tgt):
            continue

        disc = int(np.ceil(max(0.0, (1 - tgt / base_price) * 100)))
        price = int(np.floor(base_price * (1 - disc / 100)))

        # если округление скидки "перебило" маржу — откат на 1 п.п.
        if price < min_price_10pct and disc > 0:
            disc -= 1
            price = int(np.floor(base_price * (1 - disc / 100)))

        if price <= tgt and price >= min_price_10pct:
            return label, price, disc

    # не попали в топ-3 → минимальная цена для маржи 10%
    tgt = min_price_10pct
    disc = int(np.ceil(max(0.0, (1 - tgt / base_price) * 100)))
    price = int(np.floor(base_price * (1 - disc / 100)))

    if price < min_price_10pct and disc > 0:
        disc -= 1
        price = int(np.floor(base_price * (1 - disc / 100)))

    return "Не попадаем в Топ 3 по цене, мин цена для маржи 10%", price, disc


res = df.apply(
    lambda r: pick_price_and_discount(
        int(r["base_price"]),
        int(r["min_price_10pct"]),
        [int(r["tgt1"]), r["tgt2"], r["tgt3"]],
        ["Рассчитана скидка, ниже рынка", "Топ 2 по цене", "Топ 3 по цене"]
    ),
    axis=1,
    result_type="expand"
)

df["Статус"] = res[0]
df["Цена продажи (итог)"] = res[1].astype(int)
df["Скидка % (итог)"] = res[2].astype(int)

df["Маржинальность %"] = (((df["Цена продажи (итог)"] - df["cost"]) / df["Цена продажи (итог)"]) * 100).round(2)


## Summary по статусам

In [6]:
summary = (
    df.groupby("Статус")
      .agg(
          n=("Код товара","count"),
          share=("Код товара", lambda s: len(s)/len(df)),
          median_discount=("Скидка % (итог)","median"),
          p90_discount=("Скидка % (итог)", lambda s: s.quantile(0.9)),
          median_margin=("Маржинальность %","median"),
      )
      .sort_values("n", ascending=False)
)
summary

,n,share,median_discount,p90_discount,median_margin
Статус,,,,,
"Рассчитана скидка, ниже рынка",521,0.762811,19.0,33.0,37.64
Топ 2 по цене,73,0.106881,19.0,34.8,21.17
"Не попадаем в Топ 3 по цене, мин цена для маржи 10%",55,0.080527,30.0,39.0,10.68
Топ 3 по цене,34,0.049780,20.0,31.4,22.77


## Индекс цен по конкурентам (price_competitor / base_price)

In [ ]:
# mean_index — среднее значение индекса цены: competitor_price / base_price (в среднем насколько конкурент дороже/дешевле нашей цены до скидки)
# median_index — медианное значение индекса competitor_price / base_price (более устойчиво к выбросам)
# share_comp_cheaper — доля наблюдений, где конкурент дешевле нашей цены до скидки (competitor_price / base_price < 1)
# coverage — доля заполненных цен по конкуренту (сколько строк с ненулевой/не NaN ценой у этого конкурента)

idx_df = pd.DataFrame({
    "mean_index": {c: (df[c] / df["base_price"]).mean(skipna=True) for c in competitor_cols},
    "median_index": {c: (df[c] / df["base_price"]).median(skipna=True) for c in competitor_cols},
    "share_comp_cheaper": {c: ((df[c] / df["base_price"]) < 1).mean(skipna=True) for c in competitor_cols},
    "coverage": {c: df[c].notna().mean() for c in competitor_cols},
}).sort_values("mean_index")
idx_df

,mean_index,median_index,share_comp_cheaper,coverage
Ozon,0.812232,0.814480,0.651537,0.689605
Wildberries,0.824454,0.777128,0.221083,0.257687
visagehall.ru,0.829856,0.844251,0.560761,0.568082
rivegauche.ru,0.833710,0.850379,0.849195,0.893119
iledebeaute.ru,0.834169,0.858808,0.677892,0.724744
lamoda.ru,0.840997,0.844890,0.474378,0.532943
MagnitMarket,0.854540,0.888302,0.137628,0.175695
letu.ru,0.879387,0.908527,0.231332,0.282577
Megamarket,0.886412,0.856672,0.098097,0.130307
tsum.ru,0.998906,0.995330,0.183016,0.330893


## Выгрузка в Excel (5 листов)

In [8]:
out_xlsx = r"output_data/task1_result.xlsx"

export = df[[
    "Код товара","Наименование товара","Бренд","Сегмент",
    "markup","markup_rule",
    "cost","base_price",
    "min1","min1_site","min2","min2_site","min3","min3_site",
    "Цена продажи (итог)","Скидка % (итог)","Маржинальность %","Статус"
]].copy()

export = export.rename(columns={
    "Наименование товара":"Наименование",
    "markup":"Наценка (доля)",
    "markup_rule":"Правило наценки",
    "cost":"Цена закупки",
    "base_price":"Цена до скидки",
    "min1":"Мин цена рынка",
    "min1_site":"Конкурент (мин)",
    "min2":"2-я мин цена",
    "min2_site":"Конкурент (2-я)",
    "min3":"3-я мин цена",
    "min3_site":"Конкурент (3-я)",
})

s1 = export[export["Статус"] == "Рассчитана скидка, ниже рынка"]
s2 = export[export["Статус"] == "Топ 2 по цене"]
s3 = export[export["Статус"] == "Топ 3 по цене"]
s4 = export[export["Статус"] == "Не попадаем в Топ 3 по цене, мин цена для маржи 10%"]

idx_df = pd.DataFrame({
    "mean_index": {c: (df[c] / df["base_price"]).mean(skipna=True) for c in competitor_cols},
    "median_index": {c: (df[c] / df["base_price"]).median(skipna=True) for c in competitor_cols},
    "share_comp_cheaper": {c: ((df[c] / df["base_price"]) < 1).mean(skipna=True) for c in competitor_cols},
    "coverage": {c: df[c].notna().mean() for c in competitor_cols},
}).sort_values("mean_index").reset_index().rename(columns={"index":"competitor"})

with pd.ExcelWriter(out_xlsx, engine="openpyxl") as writer:
    s1.to_excel(writer, sheet_name="Лист1_Ниже_рынка", index=False)
    s2.to_excel(writer, sheet_name="Лист2_Топ2", index=False)
    s3.to_excel(writer, sheet_name="Лист3_Топ3", index=False)
    s4.to_excel(writer, sheet_name="Лист4_Мин10%", index=False)
    idx_df.to_excel(writer, sheet_name="Индекс_цен", index=False)

out_xlsx, (len(s1), len(s2), len(s3), len(s4)), idx_df.head()

('output_data/task1_result.xlsx',
 (521, 73, 34, 55),
        competitor  mean_index  median_index  share_comp_cheaper  coverage
 0            Ozon    0.812232      0.814480            0.651537  0.689605
 1     Wildberries    0.824454      0.777128            0.221083  0.257687
 2   visagehall.ru    0.829856      0.844251            0.560761  0.568082
 3   rivegauche.ru    0.833710      0.850379            0.849195  0.893119
 4  iledebeaute.ru    0.834169      0.858808            0.677892  0.724744)

## Короткая аналитическая записка

In [9]:
display(summary)

cnt = summary["n"].to_dict()
n_total = int(df["Код товара"].nunique())

note = f"""**Краткий вывод по конкурентной стратегии (маржа ≥10%)**

Всего SKU: **{n_total}**.

Разбивка по статусам:
- **Ниже рынка (min-1)** - {cnt.get('Рассчитана скидка, ниже рынка', 0)}
- **Топ‑2 по цене (2nd min - 1)** - {cnt.get('Топ 2 по цене', 0)}
- **Топ‑3 по цене (3rd min - 1)** - {cnt.get('Топ 3 по цене', 0)}
- **Не попадаем в топ‑3** - {cnt.get('Не попадаем в Топ 3 по цене, мин цена для маржи 10%', 0)}

Для SKU, которые не попадают в топ‑3 по цене при ограничении **маржа >= 10%**, я не пытался дальше снижать цену за счёт изменения наценок: 
для них рассчитана **минимальная допустимая цена** `ceil(cost/0.9)` при сохранении маржинальности 10%.
"""

print(note)

,n,share,median_discount,p90_discount,median_margin
Статус,,,,,
"Рассчитана скидка, ниже рынка",521,0.762811,19.0,33.0,37.64
Топ 2 по цене,73,0.106881,19.0,34.8,21.17
"Не попадаем в Топ 3 по цене, мин цена для маржи 10%",55,0.080527,30.0,39.0,10.68
Топ 3 по цене,34,0.049780,20.0,31.4,22.77


**Краткий вывод по конкурентной стратегии (маржа ≥10%)**

Всего SKU: **683**.

Разбивка по статусам:
- **Ниже рынка (min-1)** - 521
- **Топ‑2 по цене (2nd min - 1)** - 73
- **Топ‑3 по цене (3rd min - 1)** - 34
- **Не попадаем в топ‑3** - 55

Для SKU, которые не попадают в топ‑3 по цене при ограничении **маржа >= 10%**, я не пытался дальше снижать цену за счёт изменения наценок: 
для них рассчитана **минимальная допустимая цена** `ceil(cost/0.9)` при сохранении маржинальности 10%.



Вопросы и доп. заметки по задаче 1 возникшие в процессе выполнения:
1) Какую стратегию выбирать для товаров, цену которых не удалось поместить в топ-3 по рынку при соблюдении мин. маржинальности 10%?
Мной была выбрана стратегия минимальной цены для маржинаности 10%, учитывая что количество товаров не попавших в топ 3 составляет 8%(Можно детальнее посмотреть в Summary по статусам в столбце Share)
2) Рассматривал вариант сокращения наценки для оставшихся товаров, но в таком случае понял, что маржа будет менее 10%.
3) Появилось так же предположение касательно анамальный отклонений от цены(GP) связанное с краткосрочными промо акциями/распродажими отдельных категорий товаров, но из-за недостаточного количества данных из парсера предложений у селлеров проблематично оценить анамальность предложения и влияние на конечный выбор и подсчет цены для товара.(Возможный демпинг цены)